<h1>SI 608 Final Project</h1>

In [1]:
import pandas as pd
import os
import networkx as nx
from networkx.algorithms import bipartite, hits

from utils import * 

In [2]:
users = load_chunks("../VoterFraud2020-main/data/users")
retweets = load_chunks("../VoterFraud2020-main/data/retweets")
tweets = load_chunks("../VoterFraud2020-main/data/tweets")

<h2>Construct retweet network</h2>

In [3]:
retweets_to_users = retweets.merge(users[['user_id', 'user_active_status']], on = 'user_id')
retweets_to_users = retweets_to_users.rename(columns = {'user_id': 'rt_user_id'})

In [4]:
retweets.columns = ['tweet_id', 'rting_user_id']

In [5]:
retweets_to_tweets = tweets.merge(retweets, on = 'tweet_id', how = 'left')

In [7]:
#prototyping subset 
retweets_to_tweets = retweets_to_tweets.sample(100000)

In [29]:
G = nx.Graph()
G.add_nodes_from(retweets_to_tweets['tweet_id'], bipartite = 1)
G.add_nodes_from(retweets_to_tweets['rting_user_id'].astype(str), bipartite = 0)

G.add_edges_from([(row['rting_user_id'], row['tweet_id']) for idx, row in retweets_to_tweets.iterrows()])

In [9]:
nx.is_connected(G)

False

<h2>HITS Algorithm</h2>

In [30]:
h, a = nx.hits(G, max_iter = 500)

In [31]:
hubs = pd.DataFrame(h.items())
hubs.columns = ['id', 'hub_score']

authorities = pd.DataFrame(a.items())
authorities.columns = ['id', 'authority_score']

In [32]:
authorities.sort_values(by = 'authority_score')

,id,authority_score
467,1338871862315667456,-0.014940
1539,1327750127679889409,-0.000714
500,1336811823232921600,-0.000540
225,1332352538855747584,-0.000504
164,1333243285389914112,-0.000338
...,...,...
132388,270746819.0,0.004204
156588,1304241701687566336.0,0.004210
136046,1251981552180920320.0,0.004228
114797,909466149221519360.0,0.004238
